# prime_comments_pull

>Pull Prime Channel Comments Youtube API Vid

In [ ]:
#| default_exp prime_comments_pull

In [ ]:
#| hide
import pandas as pd
import requests
import time

#### Use sys to import api key

In [ ]:
#| hide
# # importing sys
# import sys
 
# adding Folder_2/subfolder to the system path
# sys.path.insert(0, '~/notebooks/')
 
# importing the hello
from youtube_api import API_KEY

In [ ]:
#Keys
API_KEY= API_KEY

CHANNEL_ID=  'UC2IE2CK5Cw_26wVv3hPbAgQ' # My Channel
# CHANNEL_ID='UCY8mzqqGwl5_bTpBY9qLMAA'# Andreas Kretz	UCY8mzqqGwl5_bTpBY9qLMAA	
# CHANNEL_ID='UCW2E1sGBVde5WMMxEh5CW4w' #sqlbelle	
# CHANNEL_ID='UCFVHt6gpzRxx8BsdSSYDjVQ' #becoming a data scientist
# CHANNEL_ID='UCKCCWhzBKhkw5dOA_fSV87A' # go figure
# CHANNEL_ID='UC8ENHE5xdFSwx71u3fDH5Xw' #  the primegean
# CHANNEL_ID='UC2UXDak6o7rBm23k3Vv5dww' # tina huang
# CHANNEL_ID='UCAuUUnT6oDeKwE6v1NGQxug' # TED
# CHANNEL_ID='UCWF8SqJVNlx-ctXbLswcTcA' # FoundMyFitness


# CHANNEL_ID='UCCXojfo44Qhwdg3zmOWIJ7A' #Decent Espresso Machines
# CHANNEL_ID='UCiolFxnJSOPMmV1mh9EYyIQ' # Spromethius


# CHANNEL_NAME = 'Spromethethius' #Actual channel title
# channel_name = 'the_real_spromethius'

# # CHANNEL_ID=  'UC2IE2CK5Cw_26wVv3hPbAgQ' # My Channel
CHANNEL_NAME = 'Daves Channel' #Actual channel title
channel_name = 'daves_channel'

channel_name2 = channel_name

In [ ]:
## stratascratch vid I like
# VIDEO_ID ='fklHBWow8vE'

--------------

## Get Videos Informations

In [ ]:
def get_video_details(vid_id):
    #collecting view, like, dislike, comment counts
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id="+vid_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        
        vid_deets = response_video_stats

        return vid_deets



def get_videos(df,pageToken):
    #make api call
    
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        response = requests.get(url).json()
        time.sleep(.25)


        ### statascratch example OG
        for video in response['items'][:]:
            if video['id']['kind'] == 'youtube#video': 

                vid_id = video['id']['videoId']
                vid_channel_id = video['snippet']['channelId']
                vid_deets = get_video_details(vid_id)
                vid_response = video

                df = df.append({'vid_id':vid_id,'vid_channel_id':vid_channel_id, 'vid_deets':vid_deets,'vid_response':vid_response},
                              ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
#                 pageToken = "pageToken=" + response['nextPageToken']
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            pageToken = 'no more pages'
            break
                           
            
    return df,vid_response, pageToken

# get comments

In [ ]:
def get_top_level_comments(df, pageToken):
    channel_ids = df.vid_channel_id.unique() #create series from df.vid_id:
    single_channel = channel_ids[0]
    counter= 0
    single_id = 'g4QqJoox8tc'  #for single vidoe comments pull
    while counter<3:
#         for single_channel in channel_ids:
#         #Single Video Id
#         url = 'https://www.googleapis.com/youtube/v3/commentThreads?key='+API_KEY+'&textFormat=plainText&part=snippet%2Creplies&videoId='+single_id+'&maxResults=10000'
#         Whole Channel
        
        url = 'https://www.googleapis.com/youtube/v3/commentThreads?key='+API_KEY+'&textFormat=plainText&part=snippet%2Creplies&allThreadsRelatedToChannelId='+single_channel+'&maxResults=1000&'+pageToken
#         Test 100
#         url = 'https://www.googleapis.com/youtube/v3/commentThreads?key='+API_KEY+'&textFormat=plainText&part=snippet%2Creplies&allThreadsRelatedToChannelId='+single_channel+'&maxResults=100&'+pageToken

        # Get Top Level Comments
        response = requests.get(url).json()
        time.sleep(.1)

        print(counter)
        counter += 1
        
        #reply info
        for tcomment in response['items']:
            if tcomment['kind'] == 'youtube#commentThread':
                tcresponse = tcomment
#                 response_kind = tcomment['kind']
                vid_id = tcomment['snippet']['topLevelComment']['snippet']['videoId']
                textDisplay = tcomment['snippet']['topLevelComment']['snippet']['textDisplay']
                textOriginal = tcomment['snippet']['topLevelComment']['snippet']['textOriginal']
                publishedAt = tcomment['snippet']['topLevelComment']['snippet']['publishedAt']
                tcomment_id = tcomment['snippet']['topLevelComment']['id']
                totalReplyCount = tcomment['snippet']['totalReplyCount']

                df = df.append({'vid_id':vid_id,'textDisplay':textDisplay,'textOriginal':textOriginal,
                                'tcomment_id':tcomment_id,'tcresponse':tcresponse,'totalReplyCount':totalReplyCount,'publishedAt':publishedAt},ignore_index=True)

                
                if totalReplyCount > 0:
                    try :
                        for rep in tcomment['replies']['comments']:
                            if rep['kind'] == 'youtube#comment':

                                rep_comment = rep
                                vid_id = rep['snippet']['videoId']#vid_id
                                reply_id = rep['id']
                                textDisplay = rep['snippet']['textDisplay']
                                textOriginal = rep['snippet']['textOriginal']
                                tcomment_id = rep['snippet']['parentId']
                                publishedAt = rep['snippet']['publishedAt']
                                df = df.append({'vid_id':vid_id,'textDisplay':textDisplay,'textOriginal':textOriginal, 'tcresponse':rep_comment,
                                                'reply_id':reply_id,'rep_comment':rep_comment, 'publishedAt':publishedAt},
                                               ignore_index=True)



                    except:
                        pass




        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
#                 pageToken = "pageToken=" + response['nextPageToken']
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            pageToken = 'no more pages'
            break

    return df, response, pageToken

# Main 1
>This gets all video information from a single channel

In [ ]:
# main1
if channel_name != None:
    df = pd.DataFrame()
    pageToken = ''
    while pageToken != "no more pages":
            
        df,response, pageToken = get_videos(df,pageToken)


    vid_count = len(df)    
    vid_info_csv_name = f'{channel_name}_{CHANNEL_ID}_{vid_count}_vid_infos.csv'    
    df.to_csv(vid_info_csv_name,index=False)
    print(f'Saved: {vid_info_csv_name}')    
    
else:
    print('Please add Channel Name')

print() ; df.tail(2)

Saved: daves_channel_UC2IE2CK5Cw_26wVv3hPbAgQ_23_vid_infos.csv



,vid_id,vid_channel_id,vid_deets,vid_response
21,vpZzvLnM3G8,UC2IE2CK5Cw_26wVv3hPbAgQ,"{'kind': 'youtube#videoListResponse', 'etag': ...","{'kind': 'youtube#searchResult', 'etag': 'e4_S..."
22,PAEFiJD4kc8,UC2IE2CK5Cw_26wVv3hPbAgQ,"{'kind': 'youtube#videoListResponse', 'etag': ...","{'kind': 'youtube#searchResult', 'etag': 'ZQzc..."


# Regex Video Title and DatepublishedAt

In [ ]:
def regex_vid_title_date(vid_info_csv_name):
    df = pd.read_csv(vid_info_csv_name)
    df.vid_response= df.vid_response.astype(str)

    df['title'] = df.vid_response.str.replace(r"(.*?[\'\"]title[//]?[\'\"]: [\'\"](.*?)[\'\"],.*)",r"\2", regex=True)
    df['publishedAt'] = df.vid_response.str.replace(r"(.*?[\'\"]publishedAt[//]?[\'\"]: [\'\"](.*?)[\'\"],.*)",r"\2", regex=True)
    df.to_csv(vid_info_csv_name,index=False)
    return df
df = regex_vid_title_date(vid_info_csv_name); df[['vid_id','title','publishedAt']].tail(2)

,vid_id,title,publishedAt
21,vpZzvLnM3G8,Fast aeroPress for samples,2012-12-19T22:25:20Z
22,PAEFiJD4kc8,Espresso Stovetop Gemini Express $25 With Dave,2012-11-30T19:22:50Z


# Main 2
>This grabs all comments from the channel_id in groupings of 130 pages(of comments).  

In [ ]:
# Main 2 top level comment getter
df4 = pd.DataFrame()
pageToken="" #page token start    
ct2 = 0
#test loop
# pageToken=pageToken

while pageToken != "no more pages":
# while ct2 < 2:
    df5, res, pageToken = get_top_level_comments(df,pageToken)
#     ct2+=1
    df4 = pd.concat([df4,df5])
    df4 = df4.reset_index(drop=True)

0


In [ ]:
pageToken

'no more pages'

In [ ]:
df4.tail(2)

,vid_id,vid_channel_id,vid_deets,vid_response,title,publishedAt,tcomment_id,tcresponse,textDisplay,textOriginal,totalReplyCount,rep_comment,reply_id
40,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2013-01-20T16:51:16Z,Ugy2E4vtMUBefFxODwp4AaABAg,"{'kind': 'youtube#commentThread', 'etag': 'EdK...","Great video, Dave. I got my unit at Fante's al...","Great video, Dave. I got my unit at Fante's al...",1.0,NaN,NaN
41,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2022-01-27T17:06:46Z,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",I have this issue too- the LARGE majority of l...,I have this issue too- the LARGE majority of l...,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",Ugy2E4vtMUBefFxODwp4AaABAg.8GbOBBIfXDA9XhxEAHgkFy


In [ ]:
df4.vid_id.nunique()

23

In [ ]:
df4.groupby('textOriginal').tcresponse.count().reset_index().sort_values('tcresponse',ascending=False)

,textOriginal,tcresponse
0,Doug Oldiges Fante's at 9th and cathrine PHILL...,1
10,Thank you so much for your feedback. I went a...,1
17,old video but thank you Dave!,1
16,nice. also for more precision (geekiness) try ...,1
15,maybe try putting a couple washers under one s...,1
14,i heard someone (machine engineer but unrelate...,1
13,https://youtu.be/EIOPzw0sZac\nThis video is ta...,1
12,happy to help.,1
11,exactly,1
9,Thank you for sharing! I got a used one from e...,1


In [ ]:
# df4 = df4.drop_duplicates(subset = ['textDisplay'])
print(len(df4))
comment_count = len(df4)
vid_comment_csv_name = f'{channel_name}_{CHANNEL_ID}_youtube_comments_and_vid_info_{comment_count}rows.csv'
df4.to_csv(vid_comment_csv_name, index=False)
print(f'Saved: {vid_comment_csv_name}')
# channel_name = None # make the name None so you are reminded to rename the channel next time
df4.tail(3)

42
Saved: daves_channel_UC2IE2CK5Cw_26wVv3hPbAgQ_youtube_comments_and_vid_info_42rows.csv


,vid_id,vid_channel_id,vid_deets,vid_response,title,publishedAt,tcomment_id,tcresponse,textDisplay,textOriginal,totalReplyCount,rep_comment,reply_id
39,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2013-02-08T02:22:39Z,UgyjmOsdq_3dtoSMWz54AaABAg,"{'kind': 'youtube#commentThread', 'etag': '-He...",maybe try putting a couple washers under one s...,maybe try putting a couple washers under one s...,0.0,NaN,NaN
40,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2013-01-20T16:51:16Z,Ugy2E4vtMUBefFxODwp4AaABAg,"{'kind': 'youtube#commentThread', 'etag': 'EdK...","Great video, Dave. I got my unit at Fante's al...","Great video, Dave. I got my unit at Fante's al...",1.0,NaN,NaN
41,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2022-01-27T17:06:46Z,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",I have this issue too- the LARGE majority of l...,I have this issue too- the LARGE majority of l...,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",Ugy2E4vtMUBefFxODwp4AaABAg.8GbOBBIfXDA9XhxEAHgkFy


## Regex the comments out:

In [ ]:
def regex_comments_and_authorIds(vid_comment_csv_name):    
    
#     df = pd.read_csv(vid_comment_csv_name,low_memory=False)
    df = df4
    df = df.loc[~df.tcresponse.isna()]; df.reset_index(inplace=True,drop=True)

    df.tcresponse= df.tcresponse.astype(str) 

    df['comment'] = df.tcresponse.str.replace(r"(.*?[\'\"]textDisplay[//]?[\'\"]: [\'\"](.*?)[\'\"],.*)",r"\2", regex=True)

    df['authorChannelId'] = df.tcresponse.str.replace(r"(.*?[\'\"]authorChannelId\': {\'value[//]?[\'\"]: [\'\"](.*?)[\'\"]},.*)",r"\2", regex=True)


    ######  #newly added untested

    df['authorDisplayName'] = df.tcresponse.str.replace(r"(.*?[\'\"]authorDisplayName[\'\"]: [\'\"](.*?)[\'\"],.*)",r"\2", regex=True)    

    ######
    
    
    # create column for each unique comment id
    df['com_id'] = df.tcomment_id

    #combine reply_id and tcomment_id
    df.loc[df.com_id.isna(),'com_id'] = df.loc[df.com_id.isna()].reply_id
    
    comment_count = len(df) 
    comment_csv_name = f'{channel_name2}_{CHANNEL_ID}_youtube_comments_only_{comment_count}rows.csv'
    df.to_csv(comment_csv_name,index=False)
    print(f'Saved: {channel_name2}_{CHANNEL_ID}_youtube_comments_only_{comment_count}rows.csv')
    return df
df = regex_comments_and_authorIds(df4); df.tail(3)

Saved: daves_channel_UC2IE2CK5Cw_26wVv3hPbAgQ_youtube_comments_only_19rows.csv


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,vid_id,vid_channel_id,vid_deets,vid_response,title,publishedAt,tcomment_id,tcresponse,textDisplay,textOriginal,totalReplyCount,rep_comment,reply_id,comment,authorChannelId,authorDisplayName,com_id
16,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2013-02-08T02:22:39Z,UgyjmOsdq_3dtoSMWz54AaABAg,"{'kind': 'youtube#commentThread', 'etag': '-He...",maybe try putting a couple washers under one s...,maybe try putting a couple washers under one s...,0.0,NaN,NaN,maybe try putting a couple washers under one s...,UC2IE2CK5Cw_26wVv3hPbAgQ,David Ramsey,UgyjmOsdq_3dtoSMWz54AaABAg
17,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2013-01-20T16:51:16Z,Ugy2E4vtMUBefFxODwp4AaABAg,"{'kind': 'youtube#commentThread', 'etag': 'EdK...","Great video, Dave. I got my unit at Fante's al...","Great video, Dave. I got my unit at Fante's al...",1.0,NaN,NaN,"Great video, Dave. I got my unit at Fante's al...",UCQM7S3ckBKVTvKNXA3ruS_A,drummerdaneds,Ugy2E4vtMUBefFxODwp4AaABAg
18,PAEFiJD4kc8,NaN,NaN,NaN,NaN,2022-01-27T17:06:46Z,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",I have this issue too- the LARGE majority of l...,I have this issue too- the LARGE majority of l...,NaN,"{'kind': 'youtube#comment', 'etag': 'WLHRjwOb-...",Ugy2E4vtMUBefFxODwp4AaABAg.8GbOBBIfXDA9XhxEAHgkFy,I have this issue too- the LARGE majority of l...,UCqZPnM7t043ZEBzNboiIpKw,K Richardson,Ugy2E4vtMUBefFxODwp4AaABAg.8GbOBBIfXDA9XhxEAHgkFy


### Test to make sure comments == textDisplay

In [ ]:
len(df[df.textDisplay.isna()])==len(df[df.comment == 'nan'])

True

### Test unique comment Id's are == rows in df

In [ ]:
df.com_id.nunique() == len(df)

True

-----

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()